<center><font size = "10"> Week 9 - Cells in Space <center>
<center><font size = "8">Tutorial 03: Placing and Orienting <center>

<font size='3'><font color='blue'> In this tutorial you will see how to:
    
<font size='3'><font color='blue'> - Orientate and place cells into a sphere

### Some useful functions

In [ ]:
# Useful functions
def shift_morphology(morph,shift):
    morph.soma.points[:,[0,1,2]] += shift
    for n in morph.neurites:
        n.points[:,[0,1,2]] += shift

def center_morphology(morph):
    center = morph.soma.center.copy()
    shift_morphology(morph,-center)

def rotate_morphology(morph,R):
    morph.soma.points[:,[0,1,2]] = R.dot(morph.soma.points[:,[0,1,2]].T).T
    for n in morph.neurites:
        n.points[:,[0,1,2]] = R.dot(n.points[:,[0,1,2]].T).T    

def align_morphology_Y(morph,axis):
    # Rotation matrix that aligns the Y axis [0,1,0] to an arbitrary vector b (other than Y or -Y)
    # From https://math.stackexchange.com/a/476311
    a = np.array([0,1,0])

    b = np.array(axis) # target axis
    b = b / np.linalg.norm(b) # make unit norm

    v = np.cross(a,b)     # a x b
    s = np.linalg.norm(v) # ||v|| = sin(theta)
    c = a.dot(b)          # a . b = cos(theta)

    # Skew-symmetric cross-product matrix
    V = np.zeros((3,3))
    V[0,1] = -v[2]
    V[1,0] =  v[2]
    V[0,2] =  v[1]
    V[2,0] = -v[1]
    V[1,2] = -v[0]
    V[2,1] =  v[0]

    # Rotation matrix
    R = np.diag((1.0,1.0,1.0))
    R += V
    R += V.dot(V) * (1.0 - c) / (s * s)

    # Test
    if not np.max(R.dot(a) - b) < 1E-12:
        raise ValueError('Matrix not good!')
    
    # Rotate morphology
    rotate_morphology(morph,R)

# Poorman's plotting of morphologies
def plot_morphology(morph,color = None,plane = 'XY',somaOnly = False):
    colors = ['','','black','blue','red','green'] # colors for: soma, axon, basal, apical
    planes = { 'XY': [0,1], 'XZ': [0,2], 'YZ': [1,2], 'YX': [1,0], 'ZX': [2,0], 'ZY': [2,1] }
    if color is not None:
        colors = ['','','black',color,color,color]
    if not somaOnly:
        for n in morph.neurites:
            pts = n.points[:,[0,1,2]]
            plt.scatter(pts[:,planes[plane][0]],pts[:,planes[plane][1]],s=1,c=colors[n.type.value],marker='.')
    pts = morph.soma.center
    plt.scatter(pts[planes[plane][0]],pts[planes[plane][1]],c=colors[nm.NeuriteType.soma.value],marker='D')

def plot_morphologies(morphlist,plane = 'XY',somaOnly = False):
    colors = ['red','green','blue','magenta','cyan','yellow','orange','purple','brown']
    if len(morphlist) is 1:
        colors = [None]
    if somaOnly:
        colors = ['black' for i in range(0,len(morphlist))]
    fig = plt.figure()
    fig.suptitle(plane)
    for i,m in enumerate(morphlist):
        plot_morphology(m,colors[i % len(colors)],plane = plane,somaOnly = somaOnly)

def morph_get_all_points(morph):
    pts = morph.soma.center.copy()
    for n in morph.neurites:
        pts = np.vstack((pts,n.points[:,[0,1,2]]))
    return pts

# 1. Placing and orienting morphologies in a sphere

## Sample positions and normals in a sphere

In [ ]:
import numpy as np

N = 3 # number of positions
center = [100,100,100] # um
radius = 1000 # um

# get positions in sphere
# NOTE: we must use normal sampling to get uniform points
pos = np.random.randn(3, N)
pos /= np.linalg.norm(pos, axis = 0)
pos = pos.T

# Save normals
normals = pos.copy()

# Scale and translate
pos *= radius
pos += center

pos

## Place and align cells

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import neurom as nm
morph_files = ['Pyr_01.swc','Pyr_02.swc','Int_01.swc']
morphs = [nm.load_neuron(x) for x in morph_files]

# place all morphologies
for i,m in enumerate(morphs):
    center_morphology(m)
    align_morphology_Y(m,normals[i])
    shift_morphology(m,pos[i])

[x.soma.center for x in morphs] # print centers

In [ ]:
# Visualize in all three planes

plot_morphologies(morphs,plane = 'XY')
# Plot sphere
angles = np.linspace(0,2.0 * np.pi,360)
xp = center[0] + radius * np.cos(angles)
yp = center[1] + radius * np.sin(angles)
plt.plot(xp,yp)

plot_morphologies(morphs,plane = 'XZ')
# Plot sphere
angles = np.linspace(0,2.0 * np.pi,360)
xp = center[0] + radius * np.cos(angles)
yp = center[2] + radius * np.sin(angles)
plt.plot(xp,yp)

plot_morphologies(morphs,plane = 'YZ')
# Plot sphere
angles = np.linspace(0,2.0 * np.pi,360)
xp = center[1] + radius * np.cos(angles)
yp = center[2] + radius * np.sin(angles)
plt.plot(xp,yp)

## Visualize in 3D!

In [ ]:
# This is not a good practice in Noto, so a message will pop up.
# Anyways it should work

!pip install plotly

In [ ]:
# From https://stackoverflow.com/a/53445592
# Import dependencies
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

colors = ['red','green','blue','magenta','cyan','yellow','orange','purple','brown']

data = []
for i,m in enumerate(morphs):
    pts = morph_get_all_points(m)
    # Configure the trace.
    trace = go.Scatter3d(
        x=pts[:,0],  # <-- Put your data instead
        y=pts[:,1],  # <-- Put your data instead
        z=pts[:,2],  # <-- Put your data instead
        mode='markers',
        marker={
            'size': 1,
            'opacity': 1.0,
            'color': colors[i]
        }
    )
    data.append(trace)

# Overimpose somas
somas = np.array([m.soma.center for m in morphs])
trace = go.Scatter3d(
    x=somas[:,0],  # <-- Put your data instead
    y=somas[:,1],  # <-- Put your data instead
    z=somas[:,2],  # <-- Put your data instead
    mode='markers',
    marker={
        'size': 5,
        'opacity': 1.0,
        'color': 'black'
    }
)
data.append(trace)

# Plot sphere
# From https://github.com/plotly/plotly.py/issues/141#issuecomment-62782094
theta = np.linspace(0,2*np.pi,100)
phi = np.linspace(0,np.pi,100)
x = np.outer(np.cos(theta),np.sin(phi))
y = np.outer(np.sin(theta),np.sin(phi))
z = np.outer(np.ones(100),np.cos(phi))  # note this is 2d now

trace = go.Surface(
    x=radius * x + center[0],
    y=radius * y + center[1],
    z=radius * z + center[2],
    opacity = 0.3,
    showscale = False
)
data.append(trace)
    
# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)